In [37]:
import functools
import copy
from os.path import join

In [8]:
class PBDef:
    def __init__(self):
        self.defs = {}
        self.explain = None
    def add_dirichlet(self, *args):
        self.defs.append(args)
    def draw(self, name, k, i):
        
     
    def __call__(self, fn):
        @functools.wraps(fn)
        def decorated(*args, **kwargs):
            #print "In my decorator before call, with arg %s" % self.arg
            fn(self, *args, **kwargs)
            #print "In my decorator after call, with arg %s" % self.arg
        return decorated
    
    #def set_explain(self, func):
    #    def wrap(func):
    #        def wrapper(*args, **kwargs):
    #            def toss(x,y):
    #                print '{},{}'.format(x,y)
    #            kwargs['toss']=toss
    #            func(*args, **kwargs)
    #        return wrapper
    #    self.explain = wrap(func)

obs = [(2,2,3151), (2, 1, 346), (1, 2, 310), (1, 1, 1193)]

pb = PBDef()
pb.add_dirichlet(1.0, 'toss', 2, 3)

#def pb_func(func):
#    def pb_explain(args):
#        func(*args)
#    return pb_explain

def explain(pb, o1, o2, count=1, **kwargs): 
    print o1
    pb.draw('toss', o1, 1)
    print 'there'
    if o1==1:
        pb.draw('toss', o2, 2)
    else:
        pb.draw('toss', o2, 3)

pb.set_explain(explain)

print pb.defs

for (o1, o2, count) in [obs[0]]:
    pb.explain(o1, o2, count=count)
    

[(1.0, 'toss', 2, 3)]
2


NameError: global name 'toss' is not defined

In [14]:
pb.explain(1,2)

In [19]:
class MyDecorator(object):
    def __init__(self, argument):
        self.arg = argument
    
    def __call__(self, fn):
        @functools.wraps(fn)
        def decorated(*args, **kwargs):
            print "In my decorator before call, with arg %s" % self.arg
            fn(self, *args, **kwargs)
            print "In my decorator after call, with arg %s" % self.arg
        return decorated

In [20]:
@MyDecorator("some other func!")
def some_other_function(self):
    print self.arg

In [21]:
some_other_function()

In my decorator before call, with arg some other func!
some other func!
In my decorator after call, with arg some other func!


In [22]:
def explain(pb, o1, o2, count=1, **kwargs): 
    print o1
    pb.toss(o1, 1)
    print 'there'
    if o1==1:
        pb.toss(o2, 2)
    else:
        pb.toss(o2, 3)

In [71]:
class DrawError(Exception):
    def __init__(self, a, i, k1, k2):
        self.a = a
        self.i = i
        self.k1 = k1
        self.k2 = k2
    def __str__(self):
        return 'Different draws in distribution ({}, {}):\t{} and {}'.format(self.a,self.i,self.k1,self.k2)

class DistribError(Exception):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Invalid distribution name:\t{}'.format(self.name)
    
class PBDef:
    def __init__(self, distribs, out='pypb/out1.plate'):
        self.defs = {}
        self.explain = None
        self.curr_explanation = {}
        self.curr_solution = []
        self.a = 0
        self.count=1
        for distrib in distribs:
            self.add_dirichlet(*distrib)
        self.out = out
        with open(self.out, 'w') as f:
            pass
        
    def add_dirichlet(self, priors, name, k_a, i_a):
        self.defs[name] = (priors, self.a, k_a, i_a)
        self.a += 1
        
    def draw(self, name, k, i):
        if name not in self.defs:
            raise DistribError(name)
        (_, a, k_a, i_a) = self.defs[name]
        assert i<=i_a
        assert k<=k_a
        if (a, i) in self.curr_explanation:
            k2 = self.curr_explanation[(a,i)]
            if k != k2:
                raise DrawError(a, i, k, k2)
            else:
                return
        else:
            self.curr_explanation[(a,i)] = k
     
    def __call__(self, fn):
        @functools.wraps(fn)
        def decorated(*args, **kwargs):
            self.curr_explanation = {}
            self.curr_solution = []
            for _ in fn(self, *args, **kwargs):
                self.curr_solution.append(copy.copy(self.curr_explanation))
                self.curr_explanation = {}
            self.count = kwargs['count']
            assert self.count > 0 
            self.write_solution()
        return decorated
    
    def write_solution(self):
        with open(self.out, 'a') as f:
            f.write(';'.join(
                [ '.'.join(['{},{},{}'.format(a,i,k) for (a,i),k in expl.iteritems()]) 
                 for expl in self.curr_solution]+[str(self.count)])+'\n')                
                
# test

distribs = [(1.0, 'mu', 2, 3), (1.0, 'phi', 4, 2)]


@PBDef(distribs)
def explain(pb, doc, token, count=1):
    for topic in range(2):
        pb.draw('mu', topic, doc)
        pb.draw('phi', token, topic)
        yield


In [52]:
explain(1,1,count=1)

(1, 1)
{'count': 1}


In [70]:
corpus = [
    [(0,4), (3,2)],
    [(2,1), (3,5)],
    [(0,4), (1,2)]
]

for doc,l in enumerate(corpus):
    for (token, count) in l:
        explain(doc, token, count=count)